In [ ]:
!pip install earthengine-api
import ee
import geemap
ee.Authenticate()  # Follow the instructions in the browser
ee.Initialize(project='ee-isfacool')


In [ ]:
import geemap.colormaps as cm
# Authenticate and initialize Earth Engine
ee.Authenticate()  # Run this to authenticate with your Google Account
ee.Initialize()

# Define the region of interest (Delhi boundary in this example)
roi = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2') \
        .filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))

# Filter Sentinel-2 imagery for cloud-free images in 2023
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Calculate NDVI for each image
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

ndvi_collection = sentinel2.map(calculate_ndvi)

# Get the greenest image (image with the highest mean NDVI)
greenest = ndvi_collection.qualityMosaic('NDVI')

# Define visualization parameters for NDVI
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['brown', 'yellow', 'green']
}

# Mask for green spaces (NDVI > 0.3)
green_mask = greenest.select('NDVI').gt(0.3)

# Visualize the results in an interactive map
Map = geemap.Map()
Map.centerObject(roi, 10)
Map.addLayer(roi, {'color': 'red'}, 'Region of Interest')
#Map.addLayer(greenest.select('NDVI').clip(roi), ndvi_vis, 'NDVI Map')
Map.addLayer(green_mask.selfMask().clip(roi), {'palette': 'green'}, 'Green Spaces')

# Function to display NDVI for a specific year on the map
def add_ndvi_to_map(year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    # Filter Sentinel-2 imagery
    sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .map(calculate_ndvi)

    # Calculate median NDVI image for the year
    median_image = sentinel2.median()

    # Define visualization parameters for NDVI
    ndvi_vis_params = {
        'min': 0,
        'max': 1,
        'palette': cm.palettes.ndvi  # Use a built-in NDVI palette
    }

    # Add the NDVI layer to the map
    Map.addLayer(median_image.select('NDVI').clip(roi), ndvi_vis_params, f'NDVI {year}')

# Add NDVI layers for each year to the map
for year in years:
    add_ndvi_to_map(year)

Map.addLayerControl()  # Add a layer control to toggle layers
Map

# Function to add a custom legend
def add_ndvi_legend(map_obj):
    legend_dict = {
        'Very Low': '#d73027',
        'Low': '#f46d43',
        'Moderate': '#fdae61',
        'High': '#a6d96a',
        'Very High': '#1a9850'
    }
    map_obj.add_legend(title='NDVI Legend', legend_dict=legend_dict)

# Add the legend to the map
add_ndvi_legend(Map)

# Display the updated map
Map



Map(center=[28.64654955564474, 77.10895454636062], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# Save the interactive map to an HTML file
Map.save(r'C:\Python\Exercise\urban_ndvi_analysis.html')
